# Dropbox Download Notebook
This notebook downloads the NMEA data from Dropbox, parses the multiple hourly files into a single daily file, and saves it in the correct folder.

<b>Note:</b> This notebook requires the credentials file generated in the `dropbox_credentials.ipynb` notebook to be present in the same directory.

In [2]:
# from dropbox import Dropbox
# from dropbox.files import FileMetadata, FolderMetadata
from datetime import datetime
import json
import os

The below code will ensure that the access token is valid and refresh it if necessary.

In [9]:
app_key = '2qz4xi3bzoaige3'
app_secret = 'aszgnzt0vunwr38'


def load_credentials():
    try:
        with open('dropbox_creds.json', 'r') as f:
            creds = json.load(f)
            return creds
    except FileNotFoundError:
        print("Credentials file not found. Please run the dropbox_credentials.ipynb notebook first.")
        return None


def create_dropbox_client():
    creds = load_credentials()
    if creds is None:
        return None
    access_token = creds['access_token']
    refresh_token = creds['refresh_token']
    dbx = Dropbox(app_key=app_key,
                  app_secret=app_secret,
                  oauth2_access_token=access_token,
                  oauth2_refresh_token=refresh_token)
    dbx.refresh_access_token()

    return dbx


dbx = create_dropbox_client()
if dbx is None:
    print("Failed to create Dropbox client.")
else:
    print("Successfully connected to Dropbox account:" + 
          f"{dbx.users_get_current_account().name.display_name}")

Successfully connected to Dropbox account:Ajay Quirk


In [13]:
def download_files():
    """
    Downloads all files from Dropbox into a tmp directory.
    """
    os.makedirs("tmp", exist_ok=True)

    entries = []
    result = dbx.files_list_folder("")
    entries.extend(result.entries)

    downloaded = []

    # handle pagination
    while result.has_more:
        result = dbx.files_list_folder_continue(result.cursor)
        entries.extend(result.entries)

    for entry in entries:
        if isinstance(entry, FileMetadata):
            remote_path = entry.path_lower
            fname = os.path.basename(remote_path)
            name, _ = os.path.splitext(fname)
            local_path = os.path.join("tmp", fname)

            print(f"Downloading {remote_path} -> {local_path} ...")
            dbx.files_download_to_file(local_path, remote_path)
            try:
                downloaded.append(datetime.strptime(name, "%y%m%d%H"))
            except ValueError:
                # skip this file it has an invalid date, is probably default.log
                print(f"Deleting file {fname} due to invalid date format")
                os.remove(local_path)
        else:
            print(f"Skipping folder/other file type: {entry.path_lower}")

    print("Finished Downloading!")
    return downloaded

In [14]:
files_downloaded = download_files()

Deleting file default.log due to invalid date format
Finished Downloading!


In [18]:
station_name = 'feld'

downloaded_dates = []
for file_time in files_downloaded:
    date = file_time.date()
    if date in downloaded_dates:
        continue
    downloaded_dates.append(date)
    # Get the day of year
    day_of_year = file_time.timetuple().tm_yday
    output_file_name = f"{station_name}{day_of_year}0.{date.strftime('%y')}.A"
    file_name = ""
    # with open()

feld1300.25.A
feld1310.25.A
feld1320.25.A
feld1420.25.A
